In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pytz
import geopandas as gpd
from datetime import datetime, timedelta
import scipy.io as sio
from tqdm import tqdm
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import load_model
from keras.backend import clear_session
from scipy.stats import spearmanr, rankdata
from sklearn.model_selection import train_test_split
from coastsat_keras import *

import pyfes
import os
import CoastSat_slope.SDS_slope as SDS_slope

transect_data = gpd.read_file('data_coastsat/transects_edit.geojson')
transect_data = transect_data[transect_data['id'].str.contains("usa_CA")]
def linestring_to_points(feature,line):
    return line.coords

transect_data['points'] = transect_data.apply(lambda l: linestring_to_points(l['site_id'],l['geometry']), axis=1)
site_id = transect_data['id'].to_numpy()

shore_data_new = pd.read_csv('CoastSat/All_CA_April_2023.csvx')

In [ ]:
beach_id = 'usa_CA_0011-0014'
shore_data,tide_heights,wave_data,dist_mop,spec1d_interp,freqz = read_shore_data_new(beach_id,site_id,shore_data_new)
df, df_time = create_df_for_keras(shore_data,tide_heights,wave_data,spec1d_interp,freqz,beach_id)
df_proj,mop_time = create_proj_data(beach_id,site_id)
X_all,X_train,X_val,X_test,y_all,y_train,y_val,y_test,df_mean,df_std,X_proj = create_train_val_test_data_sequential(df,df_proj)
model = read_keras_model(beach_id,site_id)

In [ ]:
# %matplotlib widget
plt.close('all')
fig, ((ax1),(ax0),(ax3)) = plt.subplots(nrows=3,ncols=1)
plt.subplots_adjust(wspace=0.01)
plt.subplots_adjust(hspace=0.15)
fig.set_size_inches(9.25, 7.5)

df_ = df.copy()
n = len(df_)
df_['dates'] = df_time

train_df_ = df_[0:int(n*0.7)]
val_df_ = df_[int(n*0.7):int(n*0.8)]
test_df_ = df_[int(n*0.8):]
df_ = df_.sort_values(by=['dates'])

ax0.axhline(y=0,color='black',alpha=0.2)
ax0.plot(df_['dates'],-1*(df_['shore_change'] - np.nanmean(df_['shore_change'])),color='black',alpha=0.3)
ax0.plot(train_df_['dates'],-1*(train_df_['shore_change'] - np.nanmean(df_['shore_change'])),color=sns.color_palette("Paired")[1],marker='.',linestyle="None")
ax0.plot(val_df_['dates'],-1*(val_df_['shore_change'] - np.nanmean(df_['shore_change'])),color=sns.color_palette("Paired")[7],marker='.',linestyle="None")
ax0.plot(test_df_['dates'],-1*(test_df_['shore_change'] - np.nanmean(df_['shore_change'])),color=sns.color_palette("Paired")[9],marker='.',linestyle="None")
ax0.set_xlim([pd.to_datetime('2000-01-01'),pd.to_datetime('2023-04-01')])
# ax0.set_xticklabels([])
ax0.text(pd.to_datetime('2002-11-01'),-55,'Satellite Observations\n(Sequential Split)',fontsize=12,color='black',alpha=0.8)
ax0.set_ylim([-60,45])
ax0.text(pd.to_datetime('2000-02-01'),36,'(b)',fontsize=12,color='k')
ax0.set_xticklabels([])
ax0.set_ylabel('MSL Beach Width (m)')
ax0.grid(axis='x')

train_df__ = pd.read_csv('figure_data/train_df.csv')
train_df__['dates'] = pd.to_datetime(train_df__['dates'], utc=True)
val_df__ = pd.read_csv('figure_data/val_df.csv')
val_df__['dates'] = pd.to_datetime(val_df__['dates'], utc=True)
test_df__ = pd.read_csv('figure_data/test_df.csv')
test_df__['dates'] = pd.to_datetime(test_df__['dates'], utc=True)

ax1.axhline(y=0,color='black',alpha=0.2)
ax1.plot(df_['dates'],(df_['shore_change'] - np.nanmean(df_['shore_change'])),color='black',alpha=0.3)
ax1.plot(train_df__['dates'],(train_df__['shore_change'] - np.nanmean(df_['shore_change'])),color=sns.color_palette("Paired")[1],marker='.',linestyle="None")
ax1.plot(val_df__['dates'],(val_df__['shore_change'] - np.nanmean(df_['shore_change'])),color=sns.color_palette("Paired")[7],marker='.',linestyle="None")
ax1.plot(test_df__['dates'],(test_df__['shore_change'] - np.nanmean(df_['shore_change'])),color=sns.color_palette("Paired")[9],marker='.',linestyle="None")
ax1.set_xlim([pd.to_datetime('2000-01-01'),pd.to_datetime('2023-04-01')])

ax1.text(pd.to_datetime('2011-01-01'),38,'Training',fontsize=12,color=sns.color_palette("Paired")[1])
ax1.text(pd.to_datetime('2013-05-05'),38,'Validation',fontsize=12,color=sns.color_palette("Paired")[7])
ax1.text(pd.to_datetime('2016-02-15'),38,'Test',fontsize=12,color=sns.color_palette("Paired")[9])
ax1.text(pd.to_datetime('2004-11-01'),38,'Satellite Observations\n(Random Split)',fontsize=12,color='black',alpha=0.8)
ax1.set_ylim([-45,60])
ax1.text(pd.to_datetime('2000-02-01'),52,'(a)',fontsize=12,color='k')
ax1.set_xticklabels([])
ax1.set_title('Observations and Deep Neural Network (DNN) Model Predictions at Torrey Pines')
ax1.grid(axis='x')

ind_mop = str(np.squeeze(np.argwhere(site_id==beach_id)) + 1)
mop_fold = '../mop/MOP_Beach_Widths/MOP_Width_'
beach_width = sio.loadmat(mop_fold + ind_mop + '.mat')    
time_width = np.squeeze(beach_width['time_width'])
time_width = pd.to_datetime(time_width-719529., unit='D')
msl_width = -1*np.squeeze(beach_width['msl_width'])
msl_width = msl_width - np.nanmean(msl_width)

f2pd = np.loadtxt('figure_data/figure_1_panel_d.txt')
first_non_nan_idx = np.where(~np.isnan(f2pd))[0][0]
f2pd[first_non_nan_idx] = np.nan

ax3.axhline(y=0,color='black',alpha=0.2)
ax3.plot(time_width,-1*msl_width,color='black')
f1pd = np.loadtxt('figure_data/figure_1_panel_d_seq.txt')
first_non_nan_idx = np.where(~np.isnan(f1pd))[0][0]
f1pd[first_non_nan_idx] = np.nan
ax3.plot(time_width,-1*f1pd,color=sns.color_palette("Paired")[5],linewidth=1)
ax3.plot(time_width,-1*f2pd,color=sns.color_palette("Paired")[3],linewidth=1)
ax3.text(pd.to_datetime('2004-04-10'),40,'DNN Random',fontsize=12,color=sns.color_palette("Paired")[3])
ax3.text(pd.to_datetime('2004-04-10'),30,'DNN Sequential',fontsize=12,color=sns.color_palette("Paired")[5])
ax3.text(pd.to_datetime('2004-04-10'),50,'Surveys',fontsize=12,color='black')
ax3.set_xlim([pd.to_datetime('2000-01-01'),pd.to_datetime('2023-04-01')])
ax3.set_ylim([-45,60])
ax3.text(pd.to_datetime('2000-02-01'),52,'(c)',fontsize=12,color='k')
ax3.grid(axis='x')

plt.savefig('figures/TPSB_predictions_sequential.pdf',bbox_inches='tight')
plt.savefig('figures/TPSB_predictions_sequential.png',bbox_inches='tight')
plt.show()

f1pd = -1*f1pd
msl_width = -1*msl_width

In [ ]:
from sklearn.metrics import r2_score

def find_closest(df_list, date):
    closest_dates = [df['dates'].iloc[np.argmin(np.abs(df['dates'] - date))] for df in df_list]
    min_diff = min([abs(date - d) for d in closest_dates])
    for i, d in enumerate(closest_dates):
        if abs(date - d) == min_diff:
            return i + 1
    return None

# Apply the function to each date in time_width
results = [find_closest([train_df_, val_df_, test_df_], date) for date in time_width.tz_localize('UTC')]

plt.close('all')
fig, ((ax1),(ax0)) = plt.subplots(nrows=1,ncols=2)
plt.subplots_adjust(wspace=0.4)
plt.subplots_adjust(hspace=0.15)
fig.set_size_inches(6.25, 3.25)

cp = [sns.color_palette("Paired")[1],sns.color_palette("Paired")[7],sns.color_palette("Paired")[9]]

idx_nan = np.isnan(f1pd)
msl_width_ = msl_width.copy()
f1pd = (f1pd - np.nanmedian(f1pd))
indices_where_one = np.where(np.array(results) == 1)[0]
ax0.scatter(x = msl_width_[indices_where_one],y = f1pd[indices_where_one], color = sns.color_palette("Paired")[1],marker='.',s=20)
indices_where_one = np.where(np.array(results) == 2)[0]
ax0.scatter(x = msl_width_[indices_where_one],y = f1pd[indices_where_one], color = sns.color_palette("Paired")[7],marker='.',s=20)
indices_where_one = np.where(np.array(results) == 3)[0]
ax0.scatter(x = msl_width_[indices_where_one],y = f1pd[indices_where_one], color = sns.color_palette("Paired")[9],marker='.',s=20)
ax0.set_aspect('equal', adjustable='box')
ax0.set_xlabel('Survey Beach Width (m)',fontsize=10)
ax0.set_title('Sequential Split',fontsize=10)

# Create a 1:1 line
x = np.linspace(start=-50, stop=50, num=1000)
y = x
ax0.plot(x, y, label='y = x', color='black')
ax0.set_xlim([-25,40])
ax0.set_ylim([-25,40])

ax0.text(33,-23,'(e)',fontsize=10)

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

idx_nan_r2 = np.isnan(f1pd[indices_where_one])
nmse_f1pd = rmse(f1pd[indices_where_one][~idx_nan_r2],msl_width[indices_where_one][~idx_nan_r2])
# nmse_f1pd_mean = rmse(np.mean(msl_width[indices_where_one][~idx_nan_r2]),msl_width[indices_where_one][~idx_nan_r2])
nmse_f1pd_mean = rmse(np.mean(f1pd[indices_where_one][~idx_nan_r2]),f1pd[indices_where_one][~idx_nan_r2])
ax0.text(-23,35,'RMSE (Test): '+ str(np.round(nmse_f1pd*100)/100)+ ' m',fontsize=10)

idx_nan_r2 = np.isnan(f1pd[indices_where_one])
skill_f1pd = 1 - nmse_f1pd**2/nmse_f1pd_mean**2

ax0.text(-23,30,'Skill (Test): '+ str(np.round(skill_f1pd*100)/100),fontsize=10)

f2pd = np.loadtxt('figure_data/figure_1_panel_d.txt')
f2pd = -1*f2pd
results_ = np.loadtxt('figure_data/figure_1_panel_scatter.txt')
indices_where_one = np.where(np.array(results_) == 1)[0]
ax1.scatter(x = msl_width_[indices_where_one],y = f2pd[indices_where_one], color = sns.color_palette("Paired")[1],marker='.',s=20)
indices_where_one = np.where(np.array(results_) == 2)[0]
ax1.scatter(x = msl_width_[indices_where_one],y = f2pd[indices_where_one], color = sns.color_palette("Paired")[7],marker='.',s=20)
indices_where_one = np.where(np.array(results_) == 3)[0]
ax1.scatter(x = msl_width_[indices_where_one],y = f2pd[indices_where_one], color = sns.color_palette("Paired")[9],marker='.',s=20)
ax1.set_aspect('equal', adjustable='box')
idx_nan_r2 = np.isnan(f2pd)
ax1.set_ylabel('DNN Beach Width (m)',fontsize=10)

idx_nan_r2 = np.isnan(f2pd[indices_where_one])
nmse_f2pd = rmse(f2pd[indices_where_one][~idx_nan_r2],msl_width[indices_where_one][~idx_nan_r2])
# nmse_f2pd_mean = rmse(np.mean(msl_width[indices_where_one][~idx_nan_r2]),msl_width[indices_where_one][~idx_nan_r2])
nmse_f2pd_mean = rmse(np.mean(f2pd[indices_where_one][~idx_nan_r2]),f2pd[indices_where_one][~idx_nan_r2])
ax1.text(-23,35,'RMSE (Test): '+ str(np.round(nmse_f2pd*100)/100)+ ' m',fontsize=10)

idx_nan_r2 = np.isnan(f2pd[indices_where_one])
skill_f2pd = 1 - nmse_f2pd**2/nmse_f2pd_mean**2

ax1.text(-23,30,'Skill (Test): '+ str(np.round(skill_f2pd*100)/100),fontsize=10)

# Create a 1:1 line
x = np.linspace(start=-50, stop=50, num=1000)
y = x
ax1.plot(x, y, label='y = x', color='black')
ax1.set_xlim([-25,40])
ax1.set_ylim([-25,40])

ax1.text(33,-23,'(d)',fontsize=10)

ax1.set_xlabel('Survey Beach Width (m)',fontsize=10)
ax1.set_title('Random Split',fontsize=10)


plt.savefig('figures/TPSB_predictions_sequential_scatter.pdf',bbox_inches='tight')
plt.show()